In [ ]:
import os
import sys
import tensorflow as tf
from IPython import get_ipython
from tensorflow import keras
from tools.job.train_job import train
from constants import *
from typing import Optional, List
import matplotlib.pyplot as plt

In [ ]:
RANDOM_STATE: int = 694_201_337
VALIDATION_SPLIT: float = 0.4
GOOGLE_DRIVE_MOUNT_LOCATION: str = '/content/drive/'
PROJECT_ROOT_LOCATION: str = f'{GOOGLE_DRIVE_MOUNT_LOCATION}MyDrive/42028/assignment-3'
ENVIRONMENT_IS_GOOGLE: bool = 'google.colab' in str(get_ipython())

In [ ]:
if ENVIRONMENT_IS_GOOGLE:
    from google.colab import drive
    drive.mount(GOOGLE_DRIVE_MOUNT_LOCATION, force_remount=True)
    !mkdir /tmp/data
    !mkdir /tmp/data/kaggle
    !mkdir /tmp/data/kaggle/ashishjangra27
    !unzip -q -u -d /tmp/data/kaggle/ashishjangra27/ /content/drive/MyDrive/42028/assignment-3/data/kaggle/ashishjangra27/face-mask-12k-images-dataset.zip
    sys.path.append(os.path.abspath(PROJECT_ROOT_LOCATION))
    if os.getcwd() != PROJECT_ROOT_LOCATION:
      os.chdir(PROJECT_ROOT_LOCATION)

    BASE_PATH: str = os.getcwd()
    BASE_INPUT_PATH: str = '/tmp'
    BASE_OUTPUT_PATH: str = os.path.join(BASE_PATH, 'models')
    CLASSIFICATION_INPUT_PATH: str = os.path.join(BASE_INPUT_PATH, 'data')
    CLASSIFICATION_OUTPUT_PATH: str = BASE_OUTPUT_PATH
else:
    BASE_PATH: str = os.getcwd()
    BASE_INPUT_PATH: str = os.path.join(BASE_PATH, 'dataset')
    BASE_OUTPUT_PATH: str = os.path.join(BASE_PATH, 'models')
    CLASSIFICATION_INPUT_PATH: str = os.path.join(BASE_INPUT_PATH, 'classification', 'fruits-360')
    CLASSIFICATION_OUTPUT_PATH: str = os.path.join(BASE_OUTPUT_PATH, 'classification', 'checkpoint')

In [ ]:
print(f'Tensorflow API Version: {tf.__version__}')
print(f'Keras API Version: {keras.__version__}')
print(f'CPUS: {tf.config.list_physical_devices("CPU")}')
print(f'GPUS: {tf.config.list_physical_devices("GPU")}')
print(f'TPUS: {tf.config.list_physical_devices("TPU")}')
print(f'CWD: {os.getcwd()}')
print(f'Base: {BASE_PATH}')
print(f'Input: {BASE_INPUT_PATH}')
print(f'Output: {BASE_OUTPUT_PATH}')
print(f'Classification Input: {CLASSIFICATION_INPUT_PATH}')
print(f'Classification Output: {CLASSIFICATION_OUTPUT_PATH}')

In [ ]:
NETWORKS: List[str] = ['vgg16', 'vgg19', 'resnet50', 'resnet50v2', 'resnet101', 'resnet101v2']
MODIFY_BASE: bool = True
EPOCHS: int = 50
RESUME: bool = True
DATASET: str = MASK_DETECTOR_ASHISH
CHECKPOINT: Optional[str] = None

for network in NETWORKS:
    DISCRIMINATOR: Optional[str] = f'Google-Colab-{network}-{"modified" if MODIFY_BASE else "baseline"}'

    print(f'-----{network}-----')

    model, history, architecture, classes, (x_train, y_train) = train(
        root_input=CLASSIFICATION_INPUT_PATH,
        root_output=CLASSIFICATION_OUTPUT_PATH,
        shape=IMAGE_SHAPE,
        epochs=EPOCHS,
        resume=RESUME,
        network=network,
        modify_base=MODIFY_BASE,
        dataset=DATASET,
        checkpoint=CHECKPOINT,
        run_discriminator=DISCRIMINATOR
    )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label=f'{network} Training accuracy')
    plt.plot(epochs, val_acc, 'b', label=f'{network} Validation accuracy')
    plt.title(f'{network} Training and validation accuracy')

    plt.figure()

    plt.plot(epochs, loss, 'bo', label=f'{network} raining Loss')
    plt.plot(epochs, val_loss, 'b', label=f'{network} Validation Loss')
    plt.title(f'{network} Training and validation loss')
    plt.legend()

    plt.show()

    model.evaluate(x_train, y_train, verbose=1)

    print(f'-----{network}-----')